# Assignment 5

In [ ]:
import numpy as np 
import pandas as pd 

from matplotlib.pyplot import subplots 
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

from tqdm import tqdm



## Implement gradient boosting from scratch:
So here, if I am right, there is two main methods : One for regression and one other for classification

### **Pseudo-Code: Gradient Boosting for Regression**

Here’s a structured and refined pseudo-code for Gradient Boosting for Regression based on your notes:

---

#### **Inputs**:
- Training data: $ \{(X_i, y_i)\}_{i=1}^n $
- Differentiable loss function: $ L(y_i, F(X)) = \frac{1}{2} (y_i - F(X_i))^2 $
- Learning rate: $ \alpha $
- Number of trees: $ M $
- Maximum tree depth: $ d $

---

#### **Step 1: Initialization**
1. Compute the **initial prediction** $ F_0 $ as the average of the target values:
   $$
   F_0 = \frac{1}{n} \sum_{i=1}^n y_i
   $$

---

#### **Step 2: Gradient Boosting Loop**
For $ m = 1 $ to $ M $ (number of trees):
1. **Compute residuals**:
   $$
   r_{m, i} = -\frac{\partial L(y_i, F_{m-1}(X_i))}{\partial F_{m-1}(X_i)}
   $$
   For $ L(y_i, F(X)) = \frac{1}{2}(y_i - F(X))^2 $, this simplifies to:
   $$
   r_{m, i} = y_i - F_{m-1}(X_i)
   $$

2. **Fit a regression tree**:
   - Train a regression tree $ T_m(X) $ of maximum depth $ d $ on the residuals $ r_{m, i} $.
   - The tree splits the data into $ J $ terminal regions (leaves) $ R_{m, j} $.

3. **Compute leaf values**:
   For each leaf $ R_{m, j} $, compute:
   $$
   \gamma_{m, j} = \text{average of residuals in } R_{m, j}
   $$
   $$
   \gamma_{m, j} = \frac{\sum_{X_i \in R_{m,j}} r_{m, i}}{|R_{m, j}|}
   $$

4. **Update the prediction model**:
   For each sample $ X_i $:
   $$
   F_m(X_i) = F_{m-1}(X_i) + \alpha \cdot \gamma_{m, j}, \quad \text{where } X_i \in R_{m,j}
   $$

---

#### **Step 3: Final Prediction**
For each sample $ X_i $, compute the final prediction:
$$
F_M(X_i) = F_0 + \sum_{m=1}^M \alpha \cdot \gamma_{m, j}, \quad \text{where } X_i \in R_{m,j}
$$

---

#### **Key Notes**
1. **Residuals**: Represent the negative gradient of the loss function with respect to the current predictions.
2. **Leaf Values**: Each leaf value $ \gamma_{m,j} $ corrects the predictions of the previous trees.
3. **Learning Rate**: $ \alpha $ controls the step size, preventing overfitting by scaling updates.
4. **Stopping Criteria**: Typically, the number of trees $ M $ or early stopping (validation loss) is used.

---

This pseudo-code is precise and matches your notes while being easy to follow. Let me know if you’d like additional examples or diagrams to illustrate the steps!


### **Pseudo-Code: Gradient Boosting for Classification**

Here’s a structured and refined pseudo-code for Gradient Boosting for Classification based on your notes:

---

#### **Inputs**:
- Training data: $ \{(X_i, y_i)\}_{i=1}^n $, where $ y_i \in \{0, 1\} $ (binary classification)
- Loss function: Negative log-likelihood:
  $$
  \mathcal J  = -y_i \ln(p) + \ln(1 + e^{F(X_i)})
  $$
- Learning rate: $ \alpha $
- Number of trees: $ M $
- Maximum tree depth: $ d $

---

#### **Step 1: Initialization**
1. Compute the **initial log-odds**:
   $$
   F_0(X_i) = \ln\left(\frac{\text{\# positive samples}}{\text{\# negative samples}}\right)
   $$
2. Compute the **initial probability** for each sample:
   $$
   p_0(X_i) = \sigma(F_0(X_i)) = \frac{1}{1 + e^{-F_0(X_i)}}
   $$

---

#### **Step 2: Gradient Boosting Loop**
For $ m = 1 $ to $ M $ (number of trees):
1. **Compute pseudo-residuals**:
   - Residuals are the negative gradient of the loss with respect to $ F(X_i) $:
     $$
     r_{m, i} = -\frac{\partial J}{\partial F(X_i)} = y_i - p_{m-1}(X_i)
     $$

2. **Fit a regression tree**:
   - Train a regression tree $ T_m(X) $ of maximum depth $ d $ on the pseudo-residuals $ r_{m, i} $.
   - The tree splits the data into $ J $ terminal regions (leaves) $ R_{m,j} $.

3. **Compute leaf values**:
   - For each leaf $ R_{m, j} $, compute:
     $$
     \gamma_{m,j} = \frac{\sum_{X_i \in R_{m,j}} r_{m, i}}{\sum_{X_i \in R_{m,j}} p_{m-1}(X_i)(1 - p_{m-1}(X_i))}
     $$
     Where:
     - $ p_{m-1}(X_i) = \sigma(F_{m-1}(X_i)) = \frac{1}{1 + e^{-F_{m-1}(X_i)}} $

4. **Update the prediction model**:
   - For each sample $ X_i $:
     $$
     F_m(X_i) = F_{m-1}(X_i) + \alpha \cdot \gamma_{m,j}, \quad \text{where } X_i \in R_{m,j}
     $$

---

#### **Step 3: Final Prediction**
1. Compute the final prediction for each sample $ X_i $:
   $$
   \hat{p}(X_i) = \sigma(F_M(X_i)) = \frac{1}{1 + e^{-F_M(X_i)}}
   $$
   Where:
   $$
   F_M(X_i) = F_0(X_i) + \sum_{m=1}^M \alpha \cdot \gamma_{m,j}, \quad \text{where } X_i \in R_{m,j}
   $$

2. Classify samples based on the probability threshold (e.g., $ \hat{p}(X_i) > 0.5 $).

---

### **Key Notes**
1. **Residuals**:
   - Residuals $ r_{m,i} $ approximate the gradient of the loss function.
   - These residuals are used to fit regression trees in each iteration.

2. **Leaf Values ($ \gamma_{m,j} $)**:
   - Computed as the ratio of the sum of residuals to the sum of the product $ p(1-p) $ within each leaf.

3. **Learning Rate ($ \alpha $)**:
   - Scales updates to prevent overfitting and stabilize learning.

4. **Final Prediction**:
   - Probabilities are derived from the final log-odds $ F_M(X_i) $ using the sigmoid function.

---

This refined pseudo-code covers all essential steps and aligns with your notes. Let me know if you'd like examples or additional clarifications!

In [ ]:
from ucimlrepo import fetch_ucirepo 
